In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("card_cust.csv")
df.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


### Q. 전처리

In [3]:
df.isna().sum()

CUST_ID                              0
BALANCE                              0
BALANCE_FREQUENCY                    0
PURCHASES                            0
ONEOFF_PURCHASES                     0
INSTALLMENTS_PURCHASES               0
CASH_ADVANCE                         0
PURCHASES_FREQUENCY                  0
ONEOFF_PURCHASES_FREQUENCY           0
PURCHASES_INSTALLMENTS_FREQUENCY     0
CASH_ADVANCE_FREQUENCY               0
CASH_ADVANCE_TRX                     0
PURCHASES_TRX                        0
CREDIT_LIMIT                         0
PAYMENTS                             0
MINIMUM_PAYMENTS                    74
PRC_FULL_PAYMENT                     0
TENURE                               0
dtype: int64

In [4]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [5]:
df.isna().sum().sum()

0

In [6]:
df_base = df.copy()

### Q1.

In [8]:
df_q1 = df[["TENURE", "BALANCE", "CREDIT_LIMIT"]].copy()
df_q1.head(2)

,TENURE,BALANCE,CREDIT_LIMIT
0,12.0,40.900749,1000.0
1,12.0,3202.467416,7000.0


In [11]:
t = 12
df_q1_sub = df_q1.loc[df_q1["TENURE"] == t, ]
df_q1_sub[["BALANCE", "CREDIT_LIMIT"]].corr()

,BALANCE,CREDIT_LIMIT
BALANCE,1.000000,0.460833
CREDIT_LIMIT,0.460833,1.000000


In [12]:
for t in df_q1["TENURE"].unique():
    df_q1_sub = df_q1.loc[df_q1["TENURE"] == t, ]
    print(round(abs(df_q1_sub[["BALANCE", "CREDIT_LIMIT"]].corr().iloc[0, 1]), 2))

0.46
0.82
0.38
0.09
0.29
0.95
0.87


In [14]:
df_corr = df_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()
df_corr.head()

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696

In [16]:
# df_corr.xs("BALANCE", level = 1) # .xs() 인덱스 기반 필터링
df_corr.xs("BALANCE", level = 1)["CREDIT_LIMIT"].abs().round(2).max()

0.95

### Q2.

In [21]:
df_q2 = df_base.drop(columns = "CUST_ID").copy()
df_q2.head(1)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.4,0.0,95.4,0.0,0.166667,0.0,0.083333,0.0,0.0,2.0,1000.0,201.802084,139.509787,0.0,12.0


In [22]:
arr_q2_nor = StandardScaler().fit_transform(df_q2)

In [24]:
arr_q2_nor[:1, ]

array([[-0.84876759, -0.41987944, -0.4419358 , -0.3740477 , -0.39530091,
        -0.48235389, -0.87270064, -0.80432139, -0.71961989, -0.68470093,
        -0.45791753, -0.56411617, -1.16166861, -0.55739615, -0.44372465,
        -0.46554357,  0.28242902]])

In [30]:
ls_k = list(range(2, 6))
k = ls_k[0]

model_kmeans = KMeans(n_clusters = k, random_state = 1234)
model_kmeans.fit(arr_q2_nor)
val_sil = silhouette_score(arr_q2_nor, labels = model_kmeans.labels_)
val_sil

0.3075281530456079

In [ ]:
ls_k = list(range(2, 6))

ls_sil = []
for k in ls_k:
    model_kmeans = KMeans(n_clusters = k, random_state = 1234)
    model_kmeans.fit(arr_q2_nor)
    val_sil = silhouette_score(arr_q2_nor, labels = model_kmeans.labels_)
    ls_sil = ls_sil + [val_sil]

In [32]:
ls_sil

[0.3075281530456079,
 0.19636128772937608,
 0.207150984946399,
 0.19274056144483248]

In [33]:
best_k = pd.Series(ls_sil, index = ls_k).idxmax()
best_k

2

In [34]:
k

5

In [ ]:
model_kmeans_best = KMeans(n_clusters = best_k, random_state = 1234)
model_kmeans_best.fit(arr_q2_nor)

In [37]:
df_q2["cluster"] = model_kmeans_best.labels_

In [38]:
df_q2["cluster"].value_counts() # 군집별 크기

cluster
0    802
1    198
Name: count, dtype: int64

In [41]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2).max()

3946.19

### Q3.

In [42]:
df_train = df_base.loc[(df_base["CUST_ID"] % 4) != 0, ]
df_test  = df_base.loc[(df_base["CUST_ID"] % 4) == 0, ]
len(df_train), len(df_test)

(752, 248)

In [44]:
model_dt = DecisionTreeRegressor(random_state = 1234)
model_dt.fit(X = df_train.drop(columns = ["CUST_ID", "ONEOFF_PURCHASES"]),
             y = df_train["ONEOFF_PURCHASES"])
pred = model_dt.predict(df_test.drop(columns = ["CUST_ID", "ONEOFF_PURCHASES"]))
pred[:4]

array([1500.,    0., 1490.,    0.])

In [52]:
y_t = df_test["ONEOFF_PURCHASES"]
y_p = pred

round(((y_t - y_p) ** 2).mean() ** 0.5, 1)

1039.2

In [45]:
from sklearn.metrics import mean_squared_error

In [53]:
round(mean_squared_error(y_true = y_t, 
                         y_pred = y_p) ** 0.5, 1)

1039.2